In [10]:
import pickle
import numpy as np
from tqdm import tqdm

In [11]:
file_name = 'data_256.pkl'
image_data, sentiment_data, sentence_data, sentence_tags, hashtag_data = pickle.load(open(file_name, 'rb'))

In [12]:
size = 3000

image_data = image_data[:size]
sentiment_data = sentiment_data[:size]
sentence_data = sentence_data[:size]
sentence_tags = sentence_tags[:size]
hashtag_data = hashtag_data[:size]

In [13]:
print(image_data.shape)
print(sentiment_data.shape)
print(sentence_data.shape)
print(len(sentence_tags))

(3000, 256, 256, 3)
(3000, 4)
(3000,)
3000


In [14]:
input_tags = [[tag[0] for tag in tags] for tags in sentence_tags]

In [15]:
from konlpy.tag import Komoran

komoran = Komoran()

output_tags = [[p[0] for p in komoran.pos(s)] for s in tqdm(sentence_data)]
[tags.insert(0, '\t') for tags in output_tags]
[tags.append('\n') for tags in output_tags]
None

100%|██████████| 3000/3000 [00:02<00:00, 1010.08it/s]


In [41]:
output_words = set()
for tags in output_tags:
    for tag in tags:
        if tag not in output_words:
            output_words.add(tag)
            
output_words = sorted(list(output_words))
output_words.insert(0, 'P')
n_output_words = len(output_words)

max_seq_length = max([len(_out) for _out in output_tags])

w2i_output = {w:i for i, w in enumerate(output_words)}

seq_list = []

for i, _out in enumerate(output_tags):
    for j in range(1, len(_out)):
        seq_list.append(_out[:j+1])
        
n_seq_data = len(seq_list)

x_word_data = np.zeros(shape=(n_seq_data, n_output_words), dtype='int32')

for i, tags in enumerate(input_tags):
    for tag in tags:
        word_data[i, w2i_output[tag]] += 1

seq_data = np.zeros(shape=(n_seq_data, max_seq_length + 1), dtype='int32')

for i, tags in enumerate(seq_list):
    seq_data[i, -len(tags):] = [w2i_output[w] for w in tags]
    
y_eye = np.eye(n_output_words)
x_seq_data = seq_data[:, :-1]
y_seq_data = y_eye[seq_data[:, -1]]

In [77]:
seq_data

array([[   0,    0,    0, ...,    0,    1, 2887],
       [   0,    0,    0, ...,    1, 2887, 6151],
       [   0,    0,    0, ..., 2887, 6151, 6751],
       ...,
       [   0,    0,    0, ..., 9770, 4304, 6569],
       [   0,    0,    0, ..., 4304, 6569, 4119],
       [   0,    0,    0, ..., 6569, 4119,    2]], dtype=int32)

In [43]:
x_word_data.shape, x_seq_data.shape, y_seq_data.shape

((129470, 9948), (129470, 147), (129470, 9948))

In [44]:
valid_size = 0
train_size = n_seq_data - valid_size
batch_size = 7000
n_batch = train_size // batch_size
n_valid = valid_size // batch_size

In [45]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM, Input, Add
from tensorflow.keras import backend as K

In [52]:
word_inputs = Input(shape=(n_output_words,))
word_dense_1 = Dense(256, activation=None)(word_inputs)
word_dense_2 = Dense(n_output_words, activation='softmax')(word_dense_1)

seq_inputs = Input(shape=(max_seq_length,))
embed = Embedding(n_output_words, 64)(seq_inputs)
lstm = LSTM(128)(embed)
seq_dense = Dense(n_output_words, activation='softmax')(lstm)

dense_sum = Add()([word_dense_2, seq_dense])
# final_dense = Dense(n_output_words, activation='softmax')(dense_sum)

model = Model([word_inputs, seq_inputs], dense_sum)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model = Sequential()
# model.add(Embedding(n_output_words, 64, input_length=max_seq_length))
# model.add(LSTM(128))
# model.add(Dense(n_output_words, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 147)]        0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 147, 64)      636672      input_20[0][0]                   
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, 9948)]       0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 128)          98816       embedding_9[0][0]                
_______________________________________________________________________________________

In [51]:
model.fit([x_word_data, x_seq_data], y_seq_data, 
          validation_split=0.1,
          batch_size=batch_size,
          epochs=1000)

Epoch 1/1000
17/17 [==============================] - 10s 588ms/step - loss: 9.0901 - accuracy: 0.0227 - val_loss: 8.5843 - val_accuracy: 0.0256
Epoch 2/1000
17/17 [==============================] - 9s 526ms/step - loss: 7.7680 - accuracy: 0.0256 - val_loss: 7.1735 - val_accuracy: 0.0247
Epoch 3/1000
17/17 [==============================] - 9s 523ms/step - loss: 7.0524 - accuracy: 0.0308 - val_loss: 7.0692 - val_accuracy: 0.0385
Epoch 4/1000
17/17 [==============================] - 9s 526ms/step - loss: 7.0122 - accuracy: 0.0364 - val_loss: 7.0493 - val_accuracy: 0.0385
Epoch 5/1000
17/17 [==============================] - 9s 524ms/step - loss: 7.0011 - accuracy: 0.0364 - val_loss: 7.0435 - val_accuracy: 0.0385
Epoch 6/1000
17/17 [==============================] - 9s 526ms/step - loss: 6.9968 - accuracy: 0.0364 - val_loss: 7.0411 - val_accuracy: 0.0385
Epoch 7/1000
17/17 [==============================] - 9s 525ms/step - loss: 6.9939 - accuracy: 0.0364 - val_loss: 7.0401 - val_accuracy

17/17 [==============================] - 9s 529ms/step - loss: 5.9205 - accuracy: 0.1435 - val_loss: 6.2775 - val_accuracy: 0.1194
Epoch 58/1000
17/17 [==============================] - 9s 531ms/step - loss: 5.8948 - accuracy: 0.1469 - val_loss: 6.2628 - val_accuracy: 0.1235
Epoch 59/1000
17/17 [==============================] - 9s 529ms/step - loss: 5.8696 - accuracy: 0.1514 - val_loss: 6.2497 - val_accuracy: 0.1247
Epoch 60/1000
17/17 [==============================] - 9s 527ms/step - loss: 5.8453 - accuracy: 0.1558 - val_loss: 6.2364 - val_accuracy: 0.1264
Epoch 61/1000
17/17 [==============================] - 9s 531ms/step - loss: 5.8212 - accuracy: 0.1597 - val_loss: 6.2237 - val_accuracy: 0.1293
Epoch 62/1000
17/17 [==============================] - 9s 526ms/step - loss: 5.7980 - accuracy: 0.1624 - val_loss: 6.2117 - val_accuracy: 0.1318
Epoch 63/1000
17/17 [==============================] - 9s 530ms/step - loss: 5.7750 - accuracy: 0.1647 - val_loss: 6.2007 - val_accuracy: 0.1324

17/17 [==============================] - 9s 530ms/step - loss: 4.9408 - accuracy: 0.2831 - val_loss: 5.9281 - val_accuracy: 0.1802
Epoch 114/1000
17/17 [==============================] - 9s 528ms/step - loss: 4.9285 - accuracy: 0.2845 - val_loss: 5.9266 - val_accuracy: 0.1805
Epoch 115/1000
17/17 [==============================] - 9s 528ms/step - loss: 4.9154 - accuracy: 0.2865 - val_loss: 5.9251 - val_accuracy: 0.1811
Epoch 116/1000
17/17 [==============================] - 9s 528ms/step - loss: 4.9037 - accuracy: 0.2889 - val_loss: 5.9213 - val_accuracy: 0.1817
Epoch 117/1000
17/17 [==============================] - 9s 526ms/step - loss: 4.8900 - accuracy: 0.2900 - val_loss: 5.9210 - val_accuracy: 0.1824
Epoch 118/1000
17/17 [==============================] - 9s 530ms/step - loss: 4.8772 - accuracy: 0.2924 - val_loss: 5.9184 - val_accuracy: 0.1823
Epoch 119/1000
17/17 [==============================] - 9s 529ms/step - loss: 4.8651 - accuracy: 0.2938 - val_loss: 5.9164 - val_accuracy: 

17/17 [==============================] - 9s 534ms/step - loss: 4.3289 - accuracy: 0.3753 - val_loss: 5.9086 - val_accuracy: 0.1868
Epoch 170/1000
17/17 [==============================] - 9s 529ms/step - loss: 4.3203 - accuracy: 0.3771 - val_loss: 5.9099 - val_accuracy: 0.1869
Epoch 171/1000
17/17 [==============================] - 9s 529ms/step - loss: 4.3108 - accuracy: 0.3793 - val_loss: 5.9074 - val_accuracy: 0.1877
Epoch 172/1000
17/17 [==============================] - 9s 530ms/step - loss: 4.3021 - accuracy: 0.3800 - val_loss: 5.9123 - val_accuracy: 0.1865
Epoch 173/1000
17/17 [==============================] - 9s 530ms/step - loss: 4.2934 - accuracy: 0.3817 - val_loss: 5.9091 - val_accuracy: 0.1866
Epoch 174/1000
17/17 [==============================] - 9s 532ms/step - loss: 4.2850 - accuracy: 0.3826 - val_loss: 5.9097 - val_accuracy: 0.1865
Epoch 175/1000
17/17 [==============================] - 9s 531ms/step - loss: 4.2773 - accuracy: 0.3837 - val_loss: 5.9113 - val_accuracy: 

17/17 [==============================] - 9s 533ms/step - loss: 3.8970 - accuracy: 0.4467 - val_loss: 5.9638 - val_accuracy: 0.1879
Epoch 226/1000
17/17 [==============================] - 9s 528ms/step - loss: 3.8896 - accuracy: 0.4469 - val_loss: 5.9640 - val_accuracy: 0.1885
Epoch 227/1000
17/17 [==============================] - 9s 530ms/step - loss: 3.8829 - accuracy: 0.4488 - val_loss: 5.9675 - val_accuracy: 0.1878
Epoch 228/1000
17/17 [==============================] - 9s 531ms/step - loss: 3.8777 - accuracy: 0.4489 - val_loss: 5.9684 - val_accuracy: 0.1881
Epoch 229/1000
17/17 [==============================] - 9s 529ms/step - loss: 3.8699 - accuracy: 0.4504 - val_loss: 5.9750 - val_accuracy: 0.1886
Epoch 230/1000
17/17 [==============================] - 9s 529ms/step - loss: 3.8638 - accuracy: 0.4515 - val_loss: 5.9731 - val_accuracy: 0.1872
Epoch 231/1000
17/17 [==============================] - 9s 531ms/step - loss: 3.8577 - accuracy: 0.4523 - val_loss: 5.9715 - val_accuracy: 

...

17/17 [==============================] - 9s 530ms/step - loss: 2.3315 - accuracy: 0.7580 - val_loss: 6.6843 - val_accuracy: 0.1653
Epoch 730/1000
17/17 [==============================] - 9s 533ms/step - loss: 2.3342 - accuracy: 0.7572 - val_loss: 6.6850 - val_accuracy: 0.1641
Epoch 731/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.3337 - accuracy: 0.7568 - val_loss: 6.6843 - val_accuracy: 0.1645
Epoch 732/1000
17/17 [==============================] - 9s 528ms/step - loss: 2.3311 - accuracy: 0.7576 - val_loss: 6.6822 - val_accuracy: 0.1651
Epoch 733/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.3280 - accuracy: 0.7588 - val_loss: 6.6879 - val_accuracy: 0.1640
Epoch 734/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.3262 - accuracy: 0.7590 - val_loss: 6.6898 - val_accuracy: 0.1643
Epoch 735/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.3248 - accuracy: 0.7597 - val_loss: 6.6884 - val_accuracy: 

17/17 [==============================] - 9s 531ms/step - loss: 2.2748 - accuracy: 0.7701 - val_loss: 6.7410 - val_accuracy: 0.1658
Epoch 786/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.2735 - accuracy: 0.7705 - val_loss: 6.7392 - val_accuracy: 0.1662
Epoch 787/1000
17/17 [==============================] - 9s 528ms/step - loss: 2.2724 - accuracy: 0.7704 - val_loss: 6.7446 - val_accuracy: 0.1660
Epoch 788/1000
17/17 [==============================] - 9s 531ms/step - loss: 2.2710 - accuracy: 0.7709 - val_loss: 6.7389 - val_accuracy: 0.1667
Epoch 789/1000
17/17 [==============================] - 9s 533ms/step - loss: 2.2700 - accuracy: 0.7714 - val_loss: 6.7426 - val_accuracy: 0.1656
Epoch 790/1000
17/17 [==============================] - 9s 530ms/step - loss: 2.2686 - accuracy: 0.7718 - val_loss: 6.7477 - val_accuracy: 0.1658
Epoch 791/1000
17/17 [==============================] - 9s 534ms/step - loss: 2.2688 - accuracy: 0.7717 - val_loss: 6.7453 - val_accuracy: 

17/17 [==============================] - 9s 531ms/step - loss: 2.2232 - accuracy: 0.7816 - val_loss: 6.8112 - val_accuracy: 0.1659
Epoch 842/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.2227 - accuracy: 0.7817 - val_loss: 6.8139 - val_accuracy: 0.1655
Epoch 843/1000
17/17 [==============================] - 9s 534ms/step - loss: 2.2209 - accuracy: 0.7825 - val_loss: 6.8159 - val_accuracy: 0.1653
Epoch 844/1000
17/17 [==============================] - 9s 533ms/step - loss: 2.2199 - accuracy: 0.7821 - val_loss: 6.8133 - val_accuracy: 0.1664
Epoch 845/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.2211 - accuracy: 0.7819 - val_loss: 6.8213 - val_accuracy: 0.1652
Epoch 846/1000
17/17 [==============================] - 9s 530ms/step - loss: 2.2317 - accuracy: 0.7781 - val_loss: 6.8181 - val_accuracy: 0.1650
Epoch 847/1000
17/17 [==============================] - 9s 530ms/step - loss: 2.2307 - accuracy: 0.7785 - val_loss: 6.8158 - val_accuracy: 

17/17 [==============================] - 9s 531ms/step - loss: 2.1787 - accuracy: 0.7910 - val_loss: 6.8845 - val_accuracy: 0.1637
Epoch 898/1000
17/17 [==============================] - 9s 534ms/step - loss: 2.1780 - accuracy: 0.7913 - val_loss: 6.8816 - val_accuracy: 0.1641
Epoch 899/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.1772 - accuracy: 0.7915 - val_loss: 6.8879 - val_accuracy: 0.1641
Epoch 900/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.1807 - accuracy: 0.7905 - val_loss: 6.8876 - val_accuracy: 0.1643
Epoch 901/1000
17/17 [==============================] - 9s 533ms/step - loss: 2.1769 - accuracy: 0.7913 - val_loss: 6.8881 - val_accuracy: 0.1641
Epoch 902/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.1744 - accuracy: 0.7923 - val_loss: 6.8890 - val_accuracy: 0.1641
Epoch 903/1000
17/17 [==============================] - 9s 531ms/step - loss: 2.1737 - accuracy: 0.7920 - val_loss: 6.8956 - val_accuracy: 

17/17 [==============================] - 9s 528ms/step - loss: 2.1388 - accuracy: 0.7994 - val_loss: 6.9463 - val_accuracy: 0.1623
Epoch 954/1000
17/17 [==============================] - 9s 530ms/step - loss: 2.1384 - accuracy: 0.7993 - val_loss: 6.9458 - val_accuracy: 0.1620
Epoch 955/1000
17/17 [==============================] - 9s 528ms/step - loss: 2.1379 - accuracy: 0.7996 - val_loss: 6.9476 - val_accuracy: 0.1619
Epoch 956/1000
17/17 [==============================] - 9s 529ms/step - loss: 2.1373 - accuracy: 0.7995 - val_loss: 6.9495 - val_accuracy: 0.1610
Epoch 957/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.1371 - accuracy: 0.7998 - val_loss: 6.9476 - val_accuracy: 0.1617
Epoch 958/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.1370 - accuracy: 0.7997 - val_loss: 6.9494 - val_accuracy: 0.1610
Epoch 959/1000
17/17 [==============================] - 9s 532ms/step - loss: 2.1373 - accuracy: 0.7994 - val_loss: 6.9472 - val_accuracy: 

In [53]:
file_model = 'model_ver6.hdf5'
model.save(file_model)

In [13]:
from tensorflow.keras.models import load_model
file_model = 'model_ver6.hdf5'
model = load_model(file_model)

In [72]:
def sequence_generation(model):
    tags = ['\t']
    sentence = ''
    while tags[-1] != '\n' and len(tags) < max_seq_length:
        x_test_word = np.zeros(shape=(1, n_output_words), dtype='int32')
        x_test_seq = np.zeros(shape=(1, max_seq_length), dtype='int32')
        x_test_seq[0, -len(tags):] = [w2i_output[w] for w in tags]
        result = model.predict([x_test_word, x_test_seq], verbose=0)
        result = np.argmax(result, axis=1)
        tags.append(output_words[result[0]])
        sentence += ' ' + tags[-1]
    return sentence.replace('\n', '')

In [78]:
start_idx = 70
end_idx = start_idx + 20
for ori, tags in zip(sentence_data[start_idx:end_idx], input_tags[start_idx:end_idx]):
    print('원본')
    print(ori[1:-1])
    print(tags)
    print('예측')
    print(sequence_generation(model))
    print()

원본
더메디닥터 클렌징 요즘 클렌징을 로 바꿨는데 피부가 더 깔끔해지고 좋아지고 있는 것 같아요 순수하고 자극이 없어 피부가 민감한분들도 사용하시는거 강추 드립니다 클렌징 하면서 피부에 영양 공급을 해준답니다 임산부 영유아 모두 사용 가능 해용 본 리뷰는 제품을 제공받아 작성 되었습니다
['더', '메', '디', '닥터', '클렌징', '요즘', '클렌징', '로', '바꾸', '피부', '더', '깔끔', '지', '좋', '지', '있', '것', '같', '순수', '자극', '없', '피부', '민감', '분', '사용', '거', '강', '추', '드리', '클렌징', '하', '피부', '영양', '공급', '하', '주', '임산부', '영유아', '모두', '사용', '가능', '해', '보', '리뷰', '제품', '제공', '받', '작성', '되']
예측
 버섯 분리수거 잊어버리 분리수거 잊어버리 정신없 졸리 졸리 탈취 탈취 다만 변색 우이 얼터밋썹 오차 마니 정류장 마늘 애매모호 화안히 말도 르와 에다가 어린이 예정 손톱 공부 공부 공부 테이블 흐트러지 서프라이즈 레몬 얼떨결 프론트 남해금산 엄정 등지 존맛탱 존맛탱 찔리 주차공간 듀크 캐비어 채 일요모오닝 일요모오닝 어째서 간직 안대 오사카 미술관 루즈핏이며 비밀 오랜만 약수역 변 송가 송가 전부 전부 전부 전부 전부 퍼렐 퍼렐 여우 여우 여우 여우 시스루 자라 려구요 상당 따라오 대량 함께 대하 메 베이직 초록 웨이팅 웨이팅 삼시세끼 제 체험 영롱 덜렁 성형 고통 클릭 너뮤좋아 놓치 하지만 하지만 손등 종로구 티 노라 블랙홀 최후의 만찬 서프라이즈 원전 담백꼬숩 인샵이다보니 새집 장도 새집 립스틱 월초 감탄 고만 유럽 갈색 밖 넷플릭스 애용 넷플릭스 후후 수다 끈 은데 하답 들르 려 넘보 로즈마리 미친 사랑 들키 레오 것 것 환상 환상 지옥 풋풋 유발 풋풋 양팔 양팔 양팔 구려 오늘 비트 원평 어네이티브

원본
예쁘다 내 새끼들
['예쁘', '내', '새끼']
예측
 버섯 분리수거 

 버섯 분리수거 잊어버리 분리수거 잊어버리 정신없 졸리 졸리 탈취 탈취 다만 변색 우이 얼터밋썹 오차 마니 정류장 마늘 애매모호 화안히 말도 르와 에다가 어린이 예정 손톱 공부 공부 공부 테이블 흐트러지 서프라이즈 레몬 얼떨결 프론트 남해금산 엄정 등지 존맛탱 존맛탱 찔리 주차공간 듀크 캐비어 채 일요모오닝 일요모오닝 어째서 간직 안대 오사카 미술관 루즈핏이며 비밀 오랜만 약수역 변 송가 송가 전부 전부 전부 전부 전부 퍼렐 퍼렐 여우 여우 여우 여우 시스루 자라 려구요 상당 따라오 대량 함께 대하 메 베이직 초록 웨이팅 웨이팅 삼시세끼 제 체험 영롱 덜렁 성형 고통 클릭 너뮤좋아 놓치 하지만 하지만 손등 종로구 티 노라 블랙홀 최후의 만찬 서프라이즈 원전 담백꼬숩 인샵이다보니 새집 장도 새집 립스틱 월초 감탄 고만 유럽 갈색 밖 넷플릭스 애용 넷플릭스 후후 수다 끈 은데 하답 들르 려 넘보 로즈마리 미친 사랑 들키 레오 것 것 환상 환상 지옥 풋풋 유발 풋풋 양팔 양팔 양팔 구려 오늘 비트 원평 어네이티브

원본
따뜻함 속 예쁜 사랑이 담긴 환한 웃음으로 함께한 유영님 커플의 함께한 기념일 촬영이 진행되었습니다 담백하게 손잡고 나란히 서서 웃고 있는 모습에 두분의 사랑과 설렘이 그대로 자연스럽게 담아집니다 사랑하는 연인과 함께하는 사진엔 언제나 그 밝음에 행복함을 담게됩니다 설렘 속 따뜻한 대화와 예쁜 웃음으로 함께한 시간 제게도 예쁘게 전달되었습니다 감스에서의 시간 기쁨의 선물 되셨길바라며 두분의 사랑 축복합니다 기쁜날 다시 또 만나요
['따뜻', '속', '예쁘', '사랑', '담기', '환하', '웃음', '함께', '유영', '커플', '함께', '기념일', '촬영', '진행', '담백', '손잡', '나란히', '서서', '웃', '있', '모습', '두', '분', '사랑', '설레', '그대로', '자연', '담', '지', '사랑', '연인', '함께', '사진', '언제나', '그', '밝', '행복', '담', '되', '설레', '속', 

['맛집', '오늘', '포인트', '이', '없', '거']

In [58]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [125]:
tf.reset_default_graph()

n_embedding = 64
n_hidden = 64
learning_rate = 1e-2

X = tf.placeholder(tf.int32, shape=(None, max_seq_length))
Y = tf.placeholder(tf.int32, shape=(None,))
# Y = tf.placeholder(tf.float32, shape=(None,))

embedding = tf.Variable(tf.random_uniform((n_output_words, n_embedding), -1, 1))
embed = tf.nn.embedding_lookup(embedding, X)

cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, embed, dtype=tf.float32)

model = tf.layers.dense(tf.reshape(outputs, [-1, max_seq_length*n_hidden]), n_output_words, activation=tf.nn.softmax)
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

predict = tf.arg_max(model, 1)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [126]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [127]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(6883, 64) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(128, 256) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(14848, 6883) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(6883,) dtype=float32_ref>]

In [128]:
total_epoch = 100

# for epoch in tqdm(range(total_epoch)):
for epoch in range(total_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
#     for batch in range(n_batch):
        batch_xs = x_train[batch*batch_size:(batch+1)*batch_size]
        batch_ys = y_train[batch*batch_size:(batch+1)*batch_size]
        
        _, loss = sess.run([optimizer, cost], feed_dict={
                X: batch_xs,
                Y: batch_ys
            })
        
        total_loss += loss
    
    total_valid_loss = 0
    for batch in range(n_valid):
        valid_xs = x_valid[batch*batch_size:(batch+1)*batch_size]
        valid_ys = y_valid[batch*batch_size:(batch+1)*batch_size]
        
        val_loss = sess.run(cost, feed_dict={
                X: valid_xs,
                Y: valid_ys
            })
        total_valid_loss += val_loss
    
    print('Epoch: %04d Cost: %.6f Valid Cost: %.6f' %(epoch + 1, total_loss / n_batch, total_valid_loss / n_valid))

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0001 Cost: 8.803528 Valid Cost: 8.801259


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0002 Cost: 8.800805 Valid Cost: 8.801259


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0003 Cost: 8.800805 Valid Cost: 8.801259


 45%|████▌     | 10/22 [00:02<00:03,  3.88it/s]


KeyboardInterrupt: 

In [115]:
def translate(tags):
    x_test = np.zeros(shape=(1, max_seq_length), dtype='float32')
    tags = tags + ['\t']
    idx = -len(tags)
    x_test[0, idx:] = [w2i_output[w] for w in tags]
    while x_test[0, -1] != w2i_output['\n'] and x_test[0][0] == 0:
        print(idx)
        pred = sess.run(predict, feed_dict={X: x_test})[0]
        x_test[0, idx-1:-1] = x_test[0, idx:]
        x_test[0, -1] = pred
        idx -= 1
    end = np.where(x_test[0]==w2i_output['\n'])[0]
    end = end[0] if end.shape[0] > 0 else -1
    ret = x_test[0, np.where(x_test[0]==w2i_output['\t'])[0][0]:end]
    return ''.join([output_words[ret]])

In [121]:
tags = input_tags[0]
x_test = np.zeros(shape=(1, max_seq_length), dtype='float32')
tags = tags + ['\t']
idx = -len(tags)
x_test[0, idx:] = [w2i_output[w] for w in tags]
while x_test[0, -1] != w2i_output['\n'] and x_test[0][0] == 0:
    print(idx)
    pred = sess.run(predict, feed_dict={X: x_test})[0]
    x_test[0, idx-1:-1] = x_test[0, idx:]
    x_test[0, -1] = pred
    idx -= 1
end = np.where(x_test[0]==w2i_output['\n'])[0]
end = end[0] if end.shape[0] > 0 else -1
ret = x_test[0, np.where(x_test[0]==w2i_output['\t'])[0][0]:end]
''.join([output_words[map(int, ret)]])

-7
-8
-9
-10
-11
-12
-13
-14
-15
-16
-17
-18
-19
-20
-21
-22
-23
-24
-25
-26
-27
-28
-29
-30
-31
-32
-33
-34
-35
-36
-37
-38
-39
-40
-41
-42
-43
-44
-45
-46
-47
-48
-49
-50
-51
-52
-53
-54
-55
-56
-57
-58
-59
-60
-61
-62
-63
-64
-65
-66
-67
-68
-69
-70
-71
-72
-73
-74
-75
-76
-77
-78
-79
-80
-81
-82
-83
-84
-85
-86
-87
-88
-89
-90
-91
-92
-93
-94
-95
-96
-97
-98
-99
-100
-101
-102
-103
-104
-105
-106
-107
-108
-109
-110
-111
-112
-113
-114
-115
-116
-117
-118


KeyboardInterrupt: 

In [124]:
''.join([output_words[int(i)] for i in ret])

'\t하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하'